In [1]:
library(dataiku)
library(dplyr)
library(data.table)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:

oneway_data <- function(x,prem,loss,pred,type = '95%', n=100){
  bs <- function(prem,loss,n=n){
    result = vector("numeric",n)
    data <- data.frame(prem,loss)
    for (i in 1:n){
      data1 <- sample_frac(data, 1.0,replace=T)
      result[i] <- sum(data1$loss)/sum(data1$prem)
    }
    if (type == '95%'){
      result <- sort(result)
      mark = floor(n*0.025)+1
      return((result[n-mark+1]-result[mark])/2)
    }else if (type == 'sd')
    return(sd(result))
    else return(0)
  }
  d <- data.frame(x = x, prem, loss, pred)
  t <- d %>% group_by(x) %>%
    summarise(prem =sum(prem),
              actual_lr = sum(loss)/sum(prem),
              pred_lr = sum(pred)/sum(prem)) %>% ungroup()
  t$sd <- setDT(d)[,bs(prem,loss,n),x]$V1
  ##ob pred to actual
  ob <- sum(t$prem*t$actual_lr) / sum(t$prem*t$pred_lr)
  t$pred_lr <- t$pred_lr * ob
  ##ob actual to 1
  ob <- sum(t$prem) / sum(t$prem*t$actual_lr)
  t$pred_lr <- t$pred_lr * ob
  t$actual_lr <- t$actual_lr * ob
  t$sd <- t$sd * ob
  return(t)
}



In [3]:
x <- c(21000, 23400, 26800)
prem <- c(0.1,0.2,0.3)
loss <- c(1000, 300, 3000)
pred <- c(700, 400, 800)

In [5]:
oneway_data(x,prem,loss,pred)

`summarise()` ungrouping output (override with `.groups` argument)



x,prem,actual_lr,pred_lr,sd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21000,0.1,1.3953488,2.2105263,0
23400,0.2,0.2093023,0.6315789,0
26800,0.3,1.3953488,0.8421053,0


In [6]:
test_df <- data.frame(x,prem,loss,pred)

oneway_data(test_df$x,test_df$prem,test_df$loss,test_df$pred)

`summarise()` ungrouping output (override with `.groups` argument)



x,prem,actual_lr,pred_lr,sd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21000,0.1,1.3953488,2.2105263,0
23400,0.2,0.2093023,0.6315789,0
26800,0.3,1.3953488,0.8421053,0
